<a href="https://colab.research.google.com/github/julmiha25-sys/Python/blob/main/3%D0%9A%D0%B5%D0%B9%D1%811_%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_%D0%BD%D0%B0_%D0%B2%D1%80%D0%B5%D0%BC%D1%8F_%D0%B2%D1%8B%D0%BF%D0%BE%D0%BB%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BA%D0%BE%D0%B4%D0%B0_5%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import ast # Для анализа деревьев синтаксического разбора кода на Python
import importlib # Для динамического импорта модулей
import sys
import inspect  # Для исследования объектов во время выполнения (кода, функций, классов, модулей)
FORBIDDEN_NAMES = ['eval'] # Запрещенные элементы по условию задания
from time import time # Для измерения времени выполнения кода
from multiprocessing import Process, Queue # Для запуска параллельных процессов и обмена данными между процессами

def check_for_malicious_code(code, output_file):
    try:
        tree = ast.parse(code) # Cоздается объект-дерево для синтаксического разбора кода

        for i in ast.walk(tree): # Обход дерева
            if isinstance(i, ast.Attribute): # Перебор атрибутов объектов
                if i.attr in FORBIDDEN_NAMES: # Поиск запрещенных элементов в атрибуте объекта
                    return f"Код содержит запрещенный атрибут: {i.attr}"

            if isinstance(i, ast.Call): # Проверка: является ли узел вызовом функции/метода
                 if isinstance(i.func, ast.Name) and i.func.id == 'eval': # Вызываемый объект - простое имя (не атрибут объекта) и он eval
                     if not i.args: # eval() без аргументов по условию задания
                         return "Код содержит вызов eval()"
        return None

    except SyntaxError as e:
        return f"Код содержит синтаксическую ошибку: {type(e).__name__}"
    except Exception as e:
        return f"Код содержит вредоносные элементы: {type(e).__name__}"

def run_tests(standard_code, user_code, test_cases, output_file):
    # Поиск модулей (файлов .py) в директории
    sys.path.insert(0, '') if '' not in sys.path else None

    try:
        # Импорт модулей
        standard_module = importlib.import_module(standard_code)
        user_module = importlib.import_module(user_code)

        # Проверка на вредоносный код с использованием inspect.getsource()
        malicious_code_message = check_for_malicious_code(user_code, "security_check.txt")

        if malicious_code_message:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write("Ошибка при проверке модуля user_malicious:\n")
                f.write(malicious_code_message)
        else:
            # Функция для выполнения пользовательского кода в отдельном процессе
            def execute_user_code(queue):
                try:
                    u_res = start_module(user_module, test_cases)
                    queue.put(('success', u_res))
                except Exception as e:
                    queue.put(('error', str(e)))

            # Создание очереди для передачи результатов между процессами
            result_queue = Queue()

            # Создание процесса для выполнения кода пользователя
            # multiprocessing.Process создает объект класса Process, который может быть запущен в отдельном процессе
            # Аргумент target принимает функцию, которую нужно выполнить в отдельном процессе
            user_process = Process(target=execute_user_code, args=(result_queue,))

            # Запуск созданного объекта процесса
            user_process.start()

            # Измерение времени начала выполнения кода
            start_time = time()

            # Ожидание завершения процесса с таймаутом 5 секунд
            user_process.join(timeout=5)

            # Измерение времени окончания выполнения кода
            end_time = time()

            if user_process.is_alive(): # Если процесс не завершился в течение 5 секунд - завершаем его
                user_process.terminate()
                user_process.join()

                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write("Предупреждение: время выполнения превышает 5 секунд.")
                return

            if not result_queue.empty():
                status, u_res = result_queue.get()
                if status == 'error':
                    with open(output_file, 'w', encoding='utf-8') as f:
                        f.write(f"Ошибка при выполнении кода пользователя: {u_res}\n")
                    return
            else:
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write("Не удалось получить результат выполнения пользовательского кода\n")
                return

            # Выполнение эталонного кода (без контроля времени)
            s_res = start_module(standard_module, test_cases)

            # Запись результатов сравнения работы функций в файл
            with open(output_file, 'w', encoding='utf-8') as f:
                # Сравниваем результаты каждого теста отдельно
                if isinstance(s_res, list) and isinstance(u_res, list):
                    # Тестируем каждый случай
                    for i, (s, u) in enumerate(zip(s_res, u_res), 1):
                        if s == u:
                            f.write(f'Тест {i} пройден\n')
                        else:
                            f.write(f'Тест {i} не пройден\n')
                            f.write(f'Ожидаемый результат: {s}\n')
                            f.write(f'Результат пользователя: {u}\n')
                else:
                    # Если не список - один кортеж
                    if s_res == u_res:
                        f.write('Тест пройден')
                    else:
                        f.write('Тест не пройден\n')
                        f.write(f'Ожидаемый результат: {s_res}\n')
                        f.write(f'Результат пользователя: {u_res}')
    except ModuleNotFoundError as e:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"Ошибка импорта модуля: {e}")

# Функция для запуска функций из модуля
def start_module(name_module, test_cases):
    # Находим первую вызываемую функцию - делаем перебор всех атрибутов модуля
    for name in dir(name_module):
        # Отфильтровываем приватные и специальные методы и проверяем, что объект можно вызвать как функцию
        if not name.startswith('_') and callable(getattr(name_module, name)):
            try:
                if test_cases:
                    # Список для хранения всех результатов
                    res = [
                        getattr(name_module, name)(*args) if isinstance(args, (list, tuple))
                        else getattr(name_module, name)(args)
                        for args in test_cases
                    ]
                    return res  # Возвращаем список всех результатов
                else:
                    return getattr(name_module, name)()  # Вызов функции без аргументов
            except Exception as e:
                print(f"Ошибка при выполнении кода пользователя: {e}")
                return None
    return None

run_tests('etalon', 'user_inf', [(1, 2), (3, 4), (5, 6)], 'output3.txt')
